In [1]:
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np
import json # library to handle JSON files

#!conda install -c conda-forge geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# # Introduction
Singapore is a international vibrant country city in Asia. Singapore is the world's most competitive economy, according to an updated global league table. It scored 84.8 out of a possible 100, beating the United States to the top spot in the ranking of 141 economies.The US, which topped the charts last year, scored 83.7. Thanks to its strategic location, reliable government and business-friendly environement, Singapore become top choice for global business enitities to put their APAC head quarters in. Expariates pick Singapore as their most wanted place for their next move because it's safe, bilingual and multinationalized. One possible and real life question firstly come up when Expariate relocate to Singapore would be,where should I live, or where should I rent my house if the accomodation is not arranged by company which should be most of the cases. Most company provides relocation package and allowance to let you choose yourself the place you want to stay. Singapore is a small island but contains 28 districts, which further splits into different areas. It's indeed a headache for those who come to Singapore for first time to choose which neighborhood is best for them. 
In this project, I will go through the data from different districts and areas using four square api to explore these neighborhood. 

# # Data
To simplify the analysis, assumption that the expatriates mostly work in CBD area or we called Core Central Region (CCR) in Singapore has been made.Depends on Expatriate profile, he or she might come to Singapore alone or with family, so a two-bed room condo unit or one-bedroom unit might be in favour to them probably. With a budget of 3000SGD, an expatriates could get a one-bedroom or just a studio in CCR region, while in Rest of Central Region (RCR), he/she could rent a two-bedroom condo unit. The commute time from RCR to CBD is within 30min by public transport or less than 20min by car, which should be acceptable. 
To start with RCR areas in Singapore, I googled "Singapore RCR" and retrieved a table of areas in each of the RCR districts.

In [3]:
source = requests.get('https://www.iproperty.com.sg/news/what-do-ccr-rcr-and-ocr-mean-in-singapore-property/').text
soup = BeautifulSoup(source, 'lxml')
table = soup.findAll('table')[1]
#print(table.prettify())
sg_df = pd.read_html(str(table))[0]
sg_df.columns = sg_df.iloc[0]
sg_df = sg_df[1:]
sg_df

,District,Area
1,3,"Alexandra Road, Tiong Bahru, Queenstown"
2,4,"Keppel, Mount Faber, Telok Blangah"
3,5,"Buona Vista, Dover, Pasir Panjang, West Coast"
4,7,"Beach Road, Bencoolen Road, Bugis, Rochor"
5,8,"Little India, Farrer Park, Serangoon Road"
6,12,"Balestier, Moulmein, Novena, Toa Payoh"
7,13,"Potong Pasir, Macpherson"
8,14,"Eunos, Geylang, Kembangan, Paya Lebar"
9,15,"Katong, Marine Parade, Siglap, Tanjong Rhu"
10,20,"Ang Mo Kio, Bishan, Braddell Road, Thomson"


These general areas will be used to explore the most popular venues through FourSquare API. Let's first seperate the areas into columns.

In [49]:
rcr_sg_df = sg_df.Area.str.split(',').apply(pd.Series).merge(sg_df.District.astype(int),left_index=True, right_index=True).melt(id_vars='District').dropna().drop(['variable'],axis=1).sort_values(by=['District'], ascending=True).rename(columns={"value":"Area"}).reset_index(drop=True)
rcr_sg_df

,District,Area
0,3,Alexandra Road
1,3,Queenstown
2,3,Tiong Bahru
3,4,Keppel
4,4,Telok Blangah
5,4,Mount Faber
6,5,Pasir Panjang
7,5,West Coast
8,5,Dover
9,5,Buona Vista
